In [1]:
import pandas as pd
import re
from sklearn.metrics import f1_score, cohen_kappa_score
from tqdm import tqdm

In [2]:
def map_fn(row):
    row = str(row)
    if(re.search('yes', row) and re.search('no', row)):
        return None
    elif(re.search('yes', row)):
        return 0
    elif(re.search('non-hateful', row)):
        return 0
    elif(re.search('no', row)):
        return 1
    elif(re.search('hateful', row)):
        return 0
    elif(re.search('hate.', row)):
        return 0

    return None

In [5]:
files = ['P_base'] + ['P1_'+str(i) for i in range(1,6)]

type = ['OS_LLMs', 'ChatGPT']
for t in type:
    for f in tqdm(files):
        df = pd.read_excel('../'+t+'/'+f+'.xlsx')

        # For FlanT5-XXL, you can choose the below 500 samples and print at the end to check it's performance to compare with ChatGPT
        # Also, keep `files = ['P_base']` and `type = ['OS_LLMs']`
        #df = df.sample(n=500, random_state=42)
        col = df.columns[2:]
        
        no_ans = []
        F1 = []
        IAA = []
        
        for i in col:
            df[i] = df[i].apply(lambda x: str(x).lower())
            
            df[i] = df[i].apply(map_fn)
    
            pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
            no_ans.append(df[i].isna().sum())
    
            tmp = df[['label', i]].dropna()
    
            F1.append(pen*f1_score(tmp['label'], tmp[i], average='weighted'))
        
            IAA.append(pen*cohen_kappa_score(tmp['label'], tmp[i]))
    
        
        res = pd.read_csv('Performance'+t+'_H.csv')
        try:
            res[f+' Hal'] = no_ans
            res[f+' F1'] = F1
            res[f+' IAA'] = IAA
        except:
            # if there is no output for 8th model (Llama3) then it should append `None`
            no_ans.append(None)
            F1.append(None)
            IAA.append(None)
            res[f+' Hal'] = no_ans
            res[f+' F1'] = F1
            res[f+' IAA'] = IAA
        res.to_csv('Performance'+t+'_H.csv', index=False)

        # To print for a single compair (FlanT5-XXL -- in case)
        # print(no_ans[-1], F1[-1], IAA[-1])

100%|██████████| 6/6 [00:00<00:00, 14.19it/s]


In [6]:
files = ['P2_'+str(i) for i in range(1,6)]
type = ['OS_LLMs', 'ChatGPT']

for t in type:
    for f in tqdm(files):
        df = pd.read_excel('../'+t+'/'+f+'.xlsx')
        #df = df.sample(n=500, random_state=42)
        col = df.columns[2:]
        
        no_ans = []
        F1 = []
        IAA = []
        
        for i in col:
            df[i] = df[i].apply(lambda x: str(x).lower())
            
            df[i] = df[i].apply(map_fn)
    
            pen = (df.shape[0]-df[i].isna().sum())/df.shape[0]
            no_ans.append(df[i].isna().sum())
    
            tmp = df[['label', i]].dropna()
    
            F1.append(pen*f1_score(tmp['label'], tmp[i], average='weighted'))
        
            IAA.append(pen*cohen_kappa_score(tmp['label'], tmp[i]))
    
        
        res = pd.read_csv('Performance'+t+'_N.csv')
        res[f+' Hal'] = no_ans
        res[f+' F1'] = F1
        res[f+' IAA'] = IAA
        res.to_csv('Performance'+t+'_N.csv', index=False)

100%|██████████| 5/5 [00:00<00:00, 18.49it/s]
